# Импорты

In [1]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import re
import nltk
from nltk.corpus import words
from nltk.metrics import edit_distance
from nltk.stem.snowball import SnowballStemmer
from spellchecker import SpellChecker
import pymorphy2
from fuzzywuzzy import fuzz
nltk.download("words")

D:\Study\ИТМО_Хакатон\venv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\mazav\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


True

# Загрузка данных

In [5]:
goods_ours = pd.read_csv('datasets/napoleon_it_retail/itmo_meta.csv', index_col='Unnamed: 0')
goods_ours.head()

,analogue_name
0,Вино RED WINE кр. п/сл. (Россия) 1L
1,Виски WILLIAM LAWSON'S купажированный алк.40% ...
2,Вода питьевая СВЯТОЙ ИСТОЧНИК природная негаз....
3,Квас 365 ДНЕЙ фильтрованный паст. ПЭТ (Россия)...
4,Кетчуп HEINZ томатный дой-пак (Россия) 320г


In [6]:
analogs = pd.read_csv('output_clean_files/analogues_clean_full.csv')
analogs.head()

,file_name,text,text_clean,feature
0,00488a41-7407-49df-a509-514fc51f2d52.jpg,"_ ПРОСТОКВАШИНО КЕФИР феа TEnA 2,5% ЭЗОГ МОЛОЧ...",ПРОСТОКВАШИНО КЕФИР феа TEnA 25 ЭЗОГ МОЛОЧНОГ...,1
1,00dfd9e2-2203-4eda-bc55-81cf2beacad0.jpg,Виски ВИЛЬЯМ лО)'СОНС 0.77 _x000C_,Виски ВИЛЬЯМ лОСОНС 0.77 _x000C_,1
2,0106df30-685e-4372-9c98-5cd10e16e7ca.jpg,"ad ‚ '. ` HOY cere! & Чоко Пай 120, ‚ —4%%\“.\...",ad . HOY cere Чоко Пай 120 4. 3 r. т wy НЕ...,1
3,0114fd9e-d1d2-4852-ba29-a2774e0aca96.jpg,"all AN Kecbup 3,2% п “о 930r. ч Casio Зеленое...",all AN Kecbup 32 п о 930r. ч Casio Зеленое Ми...,1
4,013a6e20-cc27-440e-a311-a9b944a8260b.jpg,' Набор конфет icy. | | Accop rs шоколадные ° ...,Набор конфет icy. Accop rs шоколадные 3014...,1


In [7]:
not_analogs = pd.read_csv('output_clean_files/not_analogues_clean_full.csv')
not_analogs.head()

,file_name,text,text_clean,feature
0,00c7f210-035a-4e57-ba49-6a643d07b52e.jpg,"REDDS ПИВНОЙ НАПИТОЖ 0,33Л CT/ BYT _x000C_",REDDS ПИВНОЙ НАПИТОЖ 033Л CT BYT _x000C_,0
1,00f4a973-06b5-4198-bcd1-0f15005165b2.jpg,"53M ж Сыр НОСН!_А!\!!_) sg, B _x000C_",53M ж Сыр НОСНА sg B _x000C_,0
2,010c29a1-b6c5-4455-8470-cc49555263e7.jpg,| ИМУНЕЛЕ НЕО НАПИТОК NPORYKT БЕЗ В | КИсЛОМОЛ...,ИМУНЕЛЕ НЕО НАПИТОК NPORYKT БЕЗ В КИсЛОМОЛОЧ...,0
3,01b274f8-bd81-4b9c-95e9-1c6e51b1e842.jpg,"' Напиток MIN) ВАР мд 7, | _x000C_",Напиток MIN ВАР мд 7 _x000C_,0
4,02022236-9aaf-4589-979c-e1aa53baf3df.jpg,"ВОДКА СТОЛИЧНАЯ STOLICHNAYA 40% 0,5Л | _x000C_",ВОДКА СТОЛИЧНАЯ STOLICHNAYA 40 05Л _x000C_,0


# Объединяем датасеты с аналогами и не аналогами

In [8]:
goods = pd.concat([analogs, not_analogs], axis=0).sample(frac=1).reset_index(drop=True)

In [9]:
goods.head(20)

,file_name,text,text_clean,feature
0,35dee6f3-19b1-4508-9c78-81be0bbb1e89.jpg,"ВИНО АПСНЫ КРАСНОЕ П/СЛ 0,75Л 5692° _x000C_",ВИНО АПСНЫ КРАСНОЕ ПСЛ 075Л 5692 _x000C_,1
1,35750d55-5ba3-4238-8048-3b0c15e58ead.jpg,"і — КОНФЕТЫ! ЖЕВ.МАМВА/МАМБА В ACCOPT.26,5/27,...",КОНФЕТЫ ЖЕВ.МАМВАМАМБА В ACCOPT.265275F _x000C_,0
2,cfa401c8-5596-43d3-8253-0f28415293ae.jpg,I ea . — -н о. . —ьое ол | Напиток ЛЮБИМАЯ K...,I ea . н о. . ьое ол Напиток ЛЮБИМАЯ KONA ...,0
3,d1f8be7e-7530-4c0e-81b1-9fced74ce191.jpg,| 63M MK Кефир ПРОСТОКВАШИНО 710oFr | _x000C_,63M MK Кефир ПРОСТОКВАШИНО 710oFr _x000C_,1
4,656cd957-a3c3-435a-bf6d-45423fab0e42.jpg,Ё | PEC SIOKBAWINHO КЕФИР | '}22?7.11’.?53' | ...,Ё PEC SIOKBAWINHO КЕФИР 227.11.53 MonoYy H...,1
5,adfb02bd-d92e-469f-af79-7982ba735888.jpg,"EPICA ИОГУРТ С feet од л ва , KIT) BHKOM 4.5% ...",EPICA ИОГУРТ С feet од л ва KIT BHKOM 4.5 Teo...,0
6,15a3f490-d8bb-4ecf-aa3e-4362670ebb66.jpg,"оннн ЕН ИЕ ° ПИВО СТАРЫЙ МЕЛЬНИКИЗ — \""-. БО...",оннн ЕН ИЕ ПИВО СТАРЫЙ МЕЛЬНИКИЗ . БОЧ.МЯГ...,1
7,a8e66fcc-91ae-47d5-9179-2026b320b4ef.jpg,EEE“ Виски ФОУЛЕРС 0.5n эормовой 40% _x000C_,EEE Виски ФОУЛЕРС 0.5n эормовой 40 _x000C_,0
8,aaca2797-426e-4cb2-a447-dfd361fa5094.jpg,ВН НН Р И ОННЕа аоаа аее “KA ЗИМНЯЯ ДЕРЕВЕНЬКА...,ВН НН Р И ОННЕа аоаа аее KA ЗИМНЯЯ ДЕРЕВЕНЬКА ...,0
9,3d9fe58a-784e-4952-892c-8bd3e4323d99.jpg,a ae ADREMALINE BUSH НАПИТОК SHEPTET _ МАНГО 0...,a ae ADREMALINE BUSH НАПИТОК SHEPTET МАНГО 04...,1


In [15]:
goods.to_csv("goods_full.csv", index=False)
goods.shape

(1536, 4)

# Подготавливаем текст для классификации

In [16]:
# Создаем функцию для обработки текста
def preprocess(text):
    text = text.lower()
    text = text.replace("ё", "е")
    text = text.replace("ъ", "ь")
    text = text.replace("й", "и")
    text = re.sub('[^а-яА-яa-zA-Z0-9 ]', ' ', text)
    return text

In [17]:
# Применяет функцию к столбцу датасета goods
goods['text_clean'] = goods['text_clean'].apply(preprocess)

In [18]:
# Применяет функцию к столбцу датасета goods_ours
goods_ours['analogue_name'] = goods_ours['analogue_name'].apply(preprocess)

In [19]:
goods.head()

,file_name,text,text_clean,feature
0,35dee6f3-19b1-4508-9c78-81be0bbb1e89.jpg,"ВИНО АПСНЫ КРАСНОЕ П/СЛ 0,75Л 5692° _x000C_",вино апсны красное псл 075л 5692 x000c,1
1,35750d55-5ba3-4238-8048-3b0c15e58ead.jpg,"і — КОНФЕТЫ! ЖЕВ.МАМВА/МАМБА В ACCOPT.26,5/27,...",конфеты жев мамвамамба в accopt 265275f x000c,0
2,cfa401c8-5596-43d3-8253-0f28415293ae.jpg,I ea . — -н о. . —ьое ол | Напиток ЛЮБИМАЯ K...,i ea н о ьое ол напиток любимая kona ...,0
3,d1f8be7e-7530-4c0e-81b1-9fced74ce191.jpg,| 63M MK Кефир ПРОСТОКВАШИНО 710oFr | _x000C_,63m mk кефир простоквашино 710ofr x000c,1
4,656cd957-a3c3-435a-bf6d-45423fab0e42.jpg,Ё | PEC SIOKBAWINHO КЕФИР | '}22?7.11’.?53' | ...,е pec siokbawinho кефир 227 11 53 monoyy h...,1


In [20]:
goods_ours.head()

,analogue_name
0,вино red wine кр п сл россия 1l
1,виски william lawson s купажированныи алк 40 ...
2,вода питьевая святои источник природная негаз ...
3,квас 365 днеи фильтрованныи паст пэт россия ...
4,кетчуп heinz томатныи дои пак россия 320г


# Ищем совпадения в тексте

In [21]:
threshold = 55 # Порог сходства

# Создаем датасет для сохранения аналогов
results_df = pd.DataFrame(columns=["Product1", "Product2", "Similarity"])

for index1, row1 in goods.iterrows():
    for index2, row2 in goods_ours.iterrows():
        similarity = fuzz.ratio(row1["text_clean"], row2["analogue_name"])
        if similarity >= threshold:
            results_df.loc[len(results_df)] = [row1["text_clean"], row2["analogue_name"], similarity]

# Сохраняем результаты в CSV файл
results_df.to_csv("comparison_results.csv", index=False)

In [22]:
# Сортировка результатов по столбцу "Similarity" в убывающем порядке
sorted_results_df = results_df.sort_values(by="Similarity", ascending=False)

In [23]:
sorted_results_df.head(10)

,Product1,Product2,Similarity
9,виски william lawsons купажированныи згода 40 ...,виски william lawson s купажированныи алк 40 ...,79
61,виски william lawsons купажированныи згода 40 ...,виски william lawson s купажированныи алк 40 ...,79
59,виски william lawsons купажированныи згода 40 ...,виски william lawson s купажированныи алк 40 ...,79
28,виски william lawsons купажированныи згода 40 ...,виски william lawson s купажированныи алк 40 ...,74
76,виски william lawsons купажированныи згода 40 ...,виски william lawson s купажированныи алк 40 ...,73
27,виски william lawsons купажированныи згода 40 ...,виски william lawson s купажированныи алк 40 ...,73
25,buickm william lawsons нупажированныи згода 4...,виски william lawson s купажированныи алк 40 ...,69
73,шок батончик snickers super 8095 x000c,шоколадные батончики snickers super россия 80 г,69
0,кетчуп heinz томатными ae x000c,кетчуп heinz томатныи дои пак россия 320г,69
18,вода питьевая святои источник газ пб 15л x000c,вода питьевая святои источник природная негаз ...,68


In [31]:
sorted_results_df

,Product1,Product2,Similarity
9,виски william lawsons купажированныи згода 40 ...,виски william lawson s купажированныи алк 40 ...,79
61,виски william lawsons купажированныи згода 40 ...,виски william lawson s купажированныи алк 40 ...,79
59,виски william lawsons купажированныи згода 40 ...,виски william lawson s купажированныи алк 40 ...,79
28,виски william lawsons купажированныи згода 40 ...,виски william lawson s купажированныи алк 40 ...,74
76,виски william lawsons купажированныи згода 40 ...,виски william lawson s купажированныи алк 40 ...,73
...,...,...,...
36,бзмж сметана простоквашино аи 300г 15 oo ...,сметана простоквашино 15 без змж россия 300г,55
52,старыи мельник из 7очонкд пиво свет мяг mact43...,пиво старыи мельник мягкое из бочонка светлое ...,55
63,напиток добрьыи кола tt x000c,напиток безалкогольныи добрыи кола с газ пэт ...,55
54,пиво свет балтика marko 7 алк 47 сб 0447 wad...,пиво светлое балтика классическое n3 паст алк...,55


In [24]:
sorted_results_df.iloc[0]

Product1      виски william lawsons купажированныи згода 40 ...
Product2      виски william lawson s купажированныи алк 40  ...
Similarity                                                   79
Name: 9, dtype: object

In [25]:
sorted_results_df.iloc[6]

Product1      buickm william lawsons  нупажированныи згода 4...
Product2      виски william lawson s купажированныи алк 40  ...
Similarity                                                   69
Name: 25, dtype: object

In [26]:
sorted_results_df.iloc[9]

Product1       вода питьевая святои источник газ пб 15л  x000c 
Product2      вода питьевая святои источник природная негаз ...
Similarity                                                   68
Name: 18, dtype: object

In [27]:
sorted_results_df.iloc[21]

Product1         г шок батончик snickers и super 8095 м  x000c 
Product2      шоколадные батончики snickers super  россия  80 г
Similarity                                                   65
Name: 48, dtype: object

In [28]:
sorted_results_df.iloc[14]

Product1      пиво балтика 3 классическ  св алк 4 8 жб 0 45л...
Product2      пиво светлое балтика классическое n3 паст  алк...
Similarity                                                   68
Name: 64, dtype: object

In [29]:
sorted_results_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 82 entries, 9 to 71
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Product1    82 non-null     object
 1   Product2    82 non-null     object
 2   Similarity  82 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 2.6+ KB
